In [164]:
from osgeo import gdal

In [165]:
import xarray as xr
import rasterio as rx
import numpy as np
from glob import glob
import os
from datetime import datetime as dt
from matplotlib import pyplot as plt

In [166]:
os.chdir('/disk/scratch/local.4/harry/TDX/intensity/nc')

In [167]:
asc = glob('*_A_*.nc')
desc = glob('*_D_*.nc')

In [169]:
C = 3 # coarsening factor

def open_file(F):
    ds = xr.open_dataset(F)
    intensity = [x for x in list(ds.keys()) if 'Intensity' in x]
    da = ds[intensity[0]].rename('intensity')
    coarse = da.coarsen(lat=C,lon=C,boundary='trim').mean()
    time = dt.strptime(F.split('_')[-3],'%Y%m%dT%H%M%S')
    da = coarse.assign_coords(t=time).expand_dims('t')
    return da.to_dataset().astype('float16')

def get_combined(files):
    intensity = [open_file(x) for x in files]
    LAT,LON = intensity[0].lat, intensity[0].lon
    interpolated = [ds.interp(lat=LAT,lon=LON) for ds in intensity]
    return xr.merge(interpolated)

In [170]:
ds_A = get_combined(asc)
ds_D = get_combined(desc)

In [171]:
# ds_A.to_netcdf('ascending_25looks.nc')
# ds_D.to_netcdf('descending_25looks.nc')

In [118]:
 # Cumulative Sum of Residuals